In [1]:
import pandas as pd



In [2]:
df=pd.read_csv(r"single_qna.csv\single_qna.csv")
df.head()

,QuestionType,Asin,AnswerTime,UnixTime,Question,AnswerType,Answer,Category
0,yes/no,B00004U9JP,"Jun 27, 2014",1.403852e+09,I have a 9 year old Badger 1 that needs replac...,?,I replaced my old one with this without a hitch.,Appliances
1,open-ended,B00004U9JP,"Apr 28, 2014",1.398668e+09,model number,NaN,This may help InSinkErator Model BADGER-1: Bad...,Appliances
2,yes/no,B00004U9JP,"Aug 25, 2014",1.408950e+09,can I replace Badger 1 1/3 with a Badger 5 1/2...,?,Plumbing connections will vary with different ...,Appliances
3,yes/no,B00004U9JP,"Nov 3, 2014",1.415002e+09,Does this come with power cord and dishwasher ...,?,It does not come with a power cord. It does co...,Appliances
4,open-ended,B00004U9JP,"Jun 21, 2014",1.403334e+09,loud noise inside when turned on. sounds like ...,NaN,Check if you dropped something inside.Usually ...,Appliances


In [3]:
unique_categories = list(df['Category'].unique())
categories = {cat: i for i, cat in enumerate(unique_categories)}

print(categories)


{'Appliances': 0, 'Arts Crafts and Sewing': 1, 'Automotive': 2, 'Baby': 3, 'Beauty': 4, 'Cell Phones and Accessories': 5, 'Clothing Shoes and Jewelry': 6, 'Electronics': 7, 'Grocery and Gourmet Food': 8, 'Health and Personal Care': 9, 'Home and Kitchen': 10, 'Industrial and Scientific': 11, 'Musical Instruments': 12, 'Office Products': 13, 'Patio Lawn and Garden': 14, 'Pet Supplies': 15, 'Software': 16, 'Sports and Outdoors': 17, 'Tools and Home Improvement': 18, 'Toys and Games': 19, 'Video Games': 20}


In [4]:
df[['Question','Category']].groupby(['Category']).count()

,Question
Category,
Appliances,9011
Arts Crafts and Sewing,21262
Automotive,89923
Baby,28933
Beauty,42422
Cell Phones and Accessories,85865
Clothing Shoes and Jewelry,22068
Electronics,314263
Grocery and Gourmet Food,19538


In [5]:
import plotly.express as px

fig = px.bar(df['Category'].value_counts().reset_index(), x='index', y='Category',
             labels={'index': 'Category', 'Category': 'Number of Records'},
             title='Classes vs Number of Records in each class',
             width=1500, height=500)

fig.show()


In [6]:
df['labels'] = df['Category'].map(lambda cat: categories.get(cat))


In [7]:
df.head()

,QuestionType,Asin,AnswerTime,UnixTime,Question,AnswerType,Answer,Category,labels
0,yes/no,B00004U9JP,"Jun 27, 2014",1.403852e+09,I have a 9 year old Badger 1 that needs replac...,?,I replaced my old one with this without a hitch.,Appliances,0
1,open-ended,B00004U9JP,"Apr 28, 2014",1.398668e+09,model number,NaN,This may help InSinkErator Model BADGER-1: Bad...,Appliances,0
2,yes/no,B00004U9JP,"Aug 25, 2014",1.408950e+09,can I replace Badger 1 1/3 with a Badger 5 1/2...,?,Plumbing connections will vary with different ...,Appliances,0
3,yes/no,B00004U9JP,"Nov 3, 2014",1.415002e+09,Does this come with power cord and dishwasher ...,?,It does not come with a power cord. It does co...,Appliances,0
4,open-ended,B00004U9JP,"Jun 21, 2014",1.403334e+09,loud noise inside when turned on. sounds like ...,NaN,Check if you dropped something inside.Usually ...,Appliances,0


In [8]:
df.shape

(1396896, 9)

In [26]:
import pandas as pd

selected_rows_list = [df[df['Category'] == category].head(3000) for category in unique_categories]

dataframe = pd.concat(selected_rows_list, ignore_index=True)



In [10]:
colums=['Question','labels']
df_final = dataset[colums]
df_final

,Question,labels
0,I have a 9 year old Badger 1 that needs replac...,0
1,model number,0
2,can I replace Badger 1 1/3 with a Badger 5 1/2...,0
3,Does this come with power cord and dishwasher ...,0
4,loud noise inside when turned on. sounds like ...,0
...,...,...
62995,"Why is this ""Item Under Review""? Quick questio...",20
62996,Possibly lost a part while configuring my RAT7...,20
62997,R.A.T. 5 button question??? can i customize wh...,20
62998,Scroll wheel properties: I would like to know ...,20


In [11]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(df_final, test_size=0.2, random_state=9963)

In [12]:
train_data.head()

,Question,labels
3998,Will this work with the Singer simple 3116?,1
29475,What specific bacterial strains are included i...,9
33834,"In the assembly parts bag there was a large, t...",11
29592,Once you apply this product how long does it w...,9
14008,where was it made,4


In [13]:
import torch
from transformers import AdamW

import torch.nn.functional as F

from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader

# Load the pre-trained BERT model and tokenizer
model_name, num_labels = 'bert-base-uncased', len(unique_categories)
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Prepare training data and tokenize
train_texts, train_labels = list(train_data['Question']), torch.tensor(list(train_data['labels']))
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')

# Create PyTorch dataset and DataLoader
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
batch_size, shuffle = 32, True
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)





Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
len(train_loader)

1575

In [22]:
# Fine-tune the BERT model
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print("training model on", device)
# top_batches = 1000  # Number of top batches to use for training
epoch = 0
while epoch < num_epochs:
    model.train()
    total_loss = 0
    batch_count = 0

    for batch in train_loader:
        # if batch_count >= top_batches:
        #     break  # Stop processing batches after reaching the specified number

        input_ids, attention_mask, labels = [item.to(device) for item in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        batch_loss = F.cross_entropy(logits, labels)
        batch_loss.backward()
        optimizer.step()

        total_loss += batch_loss.item()
        batch_count += 1

    print(f"Epoch {epoch + 1} - Loss: {total_loss / batch_count:.4f}")
    epoch += 1

training model on cuda
Epoch 1 - Loss: 0.2041
Epoch 2 - Loss: 0.1684
Epoch 3 - Loss: 0.1538
Epoch 4 - Loss: 0.1445
Epoch 5 - Loss: 0.1324
Epoch 6 - Loss: 0.1223
Epoch 7 - Loss: 0.1189
Epoch 8 - Loss: 0.1158
Epoch 9 - Loss: 0.1074
Epoch 10 - Loss: 0.1005


In [23]:

test_texts = list(val_data['Question'])
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

test_input_ids, test_attention_mask = map(torch.tensor, (test_encodings['input_ids'], test_encodings['attention_mask']))

test_dataset = TensorDataset(test_input_ids, test_attention_mask)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model.eval()

with torch.no_grad():
    predictions = [torch.max(model(input_ids.to(device), attention_mask=attention_mask.to(device)).logits, dim=1)[1].cpu().tolist()
                   for input_ids, attention_mask in map(lambda x: [item.to(device) for item in x], test_loader)]

prediction_list = [label for sublist in predictions for label in sublist]



In [25]:
true_labels = list(val_data['labels'])

correct_count = sum(1 for pred, true in zip(prediction_list, true_labels) if pred == true)
accuracy = correct_count / len(true_labels)

print("Number of correct elements:", correct_count)
print("Accuracy:", accuracy)


Number of correct elements: 7407
Accuracy: 0.5878571428571429
